In [20]:
import requests, pprint
from IPython.display import clear_output as co
pp = pprint.PrettyPrinter(indent=4)

class Pokemon:
    def __init__(self, name=None, abilities=None, types=None, height=None, weight=None):
        self.name = name
        if abilities is None:
            self.abilities = []
        else:
            self.abilities = abilities
        if types is None:
            self.types = []
        else:
            self.types = types
        self.height = height
        self.weight = weight
        
    # build the Pokemon object from a dictionary whose attribute names match up with dictionary's keys
    def from_dict(self, data):
        for field in ['name', 'abilities', 'types', 'height', 'weight']:
            if field in data:
                # set the attributes equal to whichever value corresponds to the key
                setattr(self, field, data[field])
                
    def __repr__(self):
        return f'<Pokemon: {self.name}>'
    
    def __str__(self):
        return f'{self.name}'

class Pokedex:
    # keep track of a list of Pokemon within the Pokedex
    _list = []
    
    @classmethod
    def instructions(self):
        print("""Type 'show' to view Pokedex.
Type 'quit' to exit the Pokedex.
Type 'sort' to view a categorized version of your Pokedex.""")
    
    # prompt for some input to add in a Pokemon's name.
    @classmethod
    def populate(self, name):
        # Check if the Pokemon we want to add has already been found in the Pokedex
        if Pokedex._list:
                for p in Pokedex._list:
                    if name.title() == p.name:
                        input('That Pokemon already exists. Try another one. ')
                        return
        try:
            # Connect to http://pokeapi.com/...
            print('Please wait while we populate the Pokedex...')
            res = requests.get(f'https://pokeapi.co/api/v2/pokemon/{name.lower()}').json()
            p = Pokemon()
            # Build the dictionary to create the new Pokemon
            data = {
                'name': res['name'].title(),
                'abilities': [a['ability']['name'].title() for a in res['abilities']],
                'types': [t['type']['name'].title() for t in res['types']],
                'height': res['height'],
                'weight': res['weight']
            }
            p.from_dict(data)
            # Add it to the list
            Pokedex._list.append(p)
        except:
            input('There was an error populating your Pokedex. Try again. ')
            
    @classmethod
    def show(self):
        return Pokedex.all()
    
    @classmethod
    def run(self):
        end = False
        input('Welcome to Python Pokedex! Press any key to continue. ')
        while not end:
            co()
            self.instructions()
            name = input('Type in the name of a Pokemon to populate your Pokedex. ').lower()
            if name == 'quit':
                end = True
                break
            elif name == 'show':
                Pokedex.show()
                input('Press any key to continue. ')
            elif name == 'sort':
                Pokedex.standardize()
                input('Press any key to continue. ')
            else:
                Pokedex.populate(name)
                
    @classmethod
    def standardize(self):
        # Build a Python dictionary that categorizes all Pokemon into any and ALL types in which they belong 
        standardized_dict = {}
        for p in Pokedex._list:
            for t in p.types:
                if t not in standardized_dict:
                    standardized_dict[t] = {}
        for p in Pokedex._list:
            # If a Pokemon has multiple types
            for t in p.types:
                # Create a Pokemon dictionary for every type they belong to. 
                if p.name not in standardized_dict[t]:
                    poke_data = {
                        p.name: {
                            'abilities': p.abilities,
                            'height': p.height,
                            'weight': p.weight
                        }
                    }
                    standardized_dict[t].update(poke_data)
                else:
                    print('That Pokemon already exists')
        pp.pprint(standardized_dict)
            
    @classmethod
    def all(self):
        if Pokedex._list:
            for i in Pokedex._list:
                print(i)

In [21]:
Pokedex.run()

Type 'show' to view Pokedex.
Type 'quit' to exit the Pokedex.
Type 'sort' to view a categorized version of your Pokedex.
Type in the name of a Pokemon to populate your Pokedex. quit
